In [1]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


# Let's make a quick call to a Frontier model to get started, as a preview!

In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]

messages


[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you! Hi!'}]

In [4]:
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

'Hi there! Nice to meet you, and welcome — I’m glad you’re here.\n\nWhat would you like to do today? If you’re unsure, here are some ideas:\n- Explain a topic you’re curious about in simple terms\n- Help with writing, editing, or proofreading (emails, essays, resumes)\n- Solve math problems, code, or work through a project\n- Plan something (a trip, event, or study plan)\n- Learn something new with a quick lesson\n- Translate or practice a language\n- Brainstorm ideas or create a story, script, or outlines\n\nTell me a prompt or your goal, and we’ll dive in. What’s on your mind right now?'

## OK onwards with our first project

In [5]:
# Let's try out this utility

ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarkyassistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [7]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)
response.choices[0].message.content

'2 + 2 equals 4.'

## And now let's build useful messages for GPT-4.1-mini, using a function

In [9]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [10]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarkyassistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. 

## Time to bring it together - the API for OpenAI is very simple!

In [11]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
summarize("https://edwarddonner.com")

'# Edward Donner\'s Website: A Playground for AI Geeks and Code Wizards\n\nWelcome to Ed’s corner of the internet, where coding, large language models (LLMs), and DJ dreams collide. Ed\'s basically the Tony Stark of AI recruiting tech, co-founding Nebula.io, a startup that’s reimagining talent hunting with some seriously fancy patented AI matching. Before this, he sold his last AI baby, untapt, so yeah, he knows his stuff.\n\nIf you’re here for AI smackdowns, check out "Outsmart," where LLMs battle it out in a little game of digital diplomacy and cunning—think *Game of Thrones* but with bots. There’s also a Connect Four for the more nostalgic crowd.\n\nNews flash! Ed’s been busy dropping nuggets on AI events and leadership through late 2025, from live AI event vibes to executive briefings and how to become an AI engineer—or maybe just impress your friends with some jargon.\n\nBasically, if you’re into AI, recruiting tech, or want to stalk a techno-DJ who codes, this is your spot. Oh, a

In [13]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://edwarddonner.com")

# Edward Donner’s Website: Where AI Nerdiness Meets DJ Daydreams

Ed’s here, coding up a storm and throwing LLMs (Large Language Models, not some obscure sports league) into battles of wit and sneakiness with games like *Connect Four* and *Outsmart*. He’s the CTO/co-founder of Nebula.io, where AI helps recruiters find your hidden talents—because apparently, AI can figure out what you’re good at better than your mom.

Highlights include a patented matching model that’s apparently a big deal (press everywhere), and a backlog of AI event recaps and leadership guides to keep you in the know through 2025. Also, Ed DJs, sorta. If that combo doesn’t scream “jack of all trades” (and a bit of chaos), I don’t know what does.

Oh, and if you want to talk shop or swap DJ playlists, he’s just an email or social media follow away.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [15]:
display_summary("https://cnn.com")

**CNN: Your One-Stop-Shop for Global Drama and Ad Complaints**

Welcome to CNN, where the news never sleeps, but your patience might—especially with all those pesky ads that either freeze, refuse to load, or blare at you like they're auditioning for a horror film. Beyond the tech tantrums, CNN covers a buffet of hot topics: from the Ukraine-Russia and Israel-Hamas conflicts to the latest on Trump and 2025 elections, sprinkled with a dash of health tips, entertainment gossip, and even climate updates for good measure.

Want more? They’ve got live TV, podcasts, newsletters, and even account settings for those who like to micromanage their news intake. Basically, CNN is your noisy, opinionated, and occasionally annoying but reliable source for everything happening on Earth (and a bit beyond). Pull up a chair, brace yourself for more ads, and enjoy the global chaos.

In [16]:
display_summary("https://anthropic.com")

# Anthropic: AI with a Conscience

Welcome to Anthropic, the AI equivalent of a thoughtful philosopher who’s also really good at coding. They're a public benefit corporation aiming to make AI safe and helpful rather than terrifying and glitchy.

**Big Deals Here:**
- *Claude Sonnet 4.5* — Apparently the "best model in the world" for agents, coding, and computer use. Someone’s confident.
- *Claude Haiku 4.5* — Because why should poetry be left out of AI advancements?
- They’re all about balancing rapid AI progress with safety and responsibility, sprinkling in some transparency, ethical commitments, and “intentional pauses” for reflection. 

In short, Anthropic builds smart AI but with the brain of a responsible adult—trying hard not to unleash Skynet anytime soon.

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI
# response =

# Step 4: print the result
# print(

In [17]:
# Step 1: Create your prompts
system_prompt = "You are an assistant that summarizes emails and suggests a subject line."
user_prompt = """
Hello,

I wanted to follow up on our last meeting and share the updated report. Please review the attached file and let me know if you have any questions.

Best regards,
John
"""

# Step 2: Make the messages list (OpenAI chat format)
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI (requires openai package and API key set up)
import openai

response = openai.chat.completions.create(
    model="gpt-4.1-mini",  # or other available model
    messages=messages
)

# Step 4: Print the result (summary or suggested subject)
print(response.choices[0].message.content)


Summary: John is following up on the previous meeting and has attached an updated report for review. He invites any questions regarding the report.

Suggested Subject Line: Follow-Up and Updated Report Attached
